In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/data_imputed_2.csv")

In [3]:
df.head()

,timestamp_local,temp,city_name,country_code,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,Gujrāt,PK,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,Gujrāt,PK,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,Gujrāt,PK,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,Gujrāt,PK,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,Gujrāt,PK,430,1782.5,60.7,2.7,533.7,377.00,253.3


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
data = df[['co']]

In [6]:
from sklearn.preprocessing import MinMaxScaler
values = data.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [7]:
scaled = scaled.reshape((19706,))

In [8]:
scaled.shape

(19706,)

In [9]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [10]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

X_max = X_train.max()
X_min = X_train.min()

In [11]:
batchSize = 16
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [12]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

C:\Users\Hassan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 166, 8)              │              32 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1328)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 168)                 │         223,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,304 (872.28 KB)

 Trainable params: 223,304 (872.28 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0027 - root_mean_squared_error: 0.0518 - val_loss: 0.0019 - val_root_mean_squared_error: 0.0437 - learning_rate: 0.0010
Epoch 2/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0019 - root_mean_squared_error: 0.0440 - val_loss: 0.0019 - val_root_mean_squared_error: 0.0435 - learning_rate: 0.0010
Epoch 3/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0019 - root_mean_squared_error: 0.0429 - val_loss: 0.0018 - val_root_mean_squared_error: 0.0421 - learning_rate: 0.0010
Epoch 4/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0018 - root_mean_squared_error: 0.0429 - val_loss: 0.0017 - val_root_mean_squared_error: 0.0409 - learning_rate: 0.0010
Epoch 5/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0017 - root_mean_squared_error: 0.0412 - val_loss: 0.0017 - val_root_mean_squared_error: 0.0406 - learning_rate: 0.0010
Epoch 6/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0016 - root_mean_squared_err

In [13]:
forecast = model.predict(X_test)

122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [14]:
poll = np.array(df["co"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [15]:
from sklearn.metrics import mean_squared_error

In [16]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 13.425984


In [17]:
# Calculate MAPE
mape = np.mean(np.abs((y_test_true - testPredict) / y_test_true)) * 100
print("Test MAPE =", mape)

Test MAPE = 1.8594613298773766


In [18]:
# Calculate accuracy as 1 - MAPE
accuracy = 100 - mape
print("Accuracy =", accuracy, "%")

Accuracy = 98.14053867012262 %


In [23]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE
mae = mean_absolute_error(y_test_true, testPredict)
print("Test MAE =", mae)

Test MAE = 7.1278114


In [19]:
forecast

array([[0.03694756, 0.03317951, 0.02794226, ..., 0.03766806, 0.03923149,
        0.03445485],
       [0.0138067 , 0.00865128, 0.00746845, ..., 0.02738076, 0.02239216,
        0.01892924],
       [0.0134774 , 0.00913968, 0.00722151, ..., 0.01842717, 0.0193835 ,
        0.02048876],
       ...,
       [0.0702417 , 0.07939713, 0.0899758 , ..., 0.11903349, 0.10806515,
        0.09589876],
       [0.187272  , 0.16975799, 0.14258707, ..., 0.18327433, 0.18218207,
        0.17423314],
       [0.02621881, 0.02848161, 0.03056005, ..., 0.01186379, 0.01777488,
        0.02196932]], dtype=float32)

In [20]:
testPredict

array([[361.13   , 359.8173 , 357.99277, ..., 361.381  , 361.92566,
        360.2616 ],
       [353.0683 , 351.27225, 350.8602 , ..., 357.79715, 356.05927,
        354.85284],
       [352.95355, 351.4424 , 350.77417, ..., 354.67795, 355.0111 ,
        355.39615],
       ...,
       [372.72888, 375.91843, 379.60376, ..., 389.72678, 385.90567,
        381.66718],
       [413.49942, 407.39795, 397.93225, ..., 412.10672, 411.72623,
        408.957  ],
       [357.39236, 358.18066, 358.90475, ..., 352.39142, 354.4507 ,
        355.91196]], dtype=float32)

In [21]:
y_test_true

array([[355.572  , 356.3136 , 357.06158, ..., 362.99432, 360.7056 ,
        358.4169 ],
       [350.94345, 350.93066, 350.91788, ..., 353.89062, 352.66318,
        351.42932],
       [354.12717, 353.02118, 351.9152 , ..., 353.60294, 353.7308 ,
        353.85226],
       ...,
       [371.7272 , 375.416  , 379.11118, ..., 498.09204, 476.279  ,
        454.46594],
       [405.86606, 399.55612, 392.24887, ..., 401.80008, 404.67053,
        404.44678],
       [357.58582, 358.25708, 358.92834, ..., 352.70792, 355.59756,
        358.48083]], dtype=float32)

In [44]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [45]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 3.1780
RMSE for segment 2: 4.4285
RMSE for segment 3: 5.5976
RMSE for segment 4: 6.4963
RMSE for segment 5: 7.2847
RMSE for segment 6: 7.9633
RMSE for segment 7: 8.5341
RMSE for segment 8: 8.7907
RMSE for segment 9: 9.1045
RMSE for segment 10: 9.2716
RMSE for segment 11: 9.3965
RMSE for segment 12: 9.4698
RMSE for segment 13: 9.6623
RMSE for segment 14: 9.9495
RMSE for segment 15: 10.3346
RMSE for segment 16: 10.2894
RMSE for segment 17: 10.1561
RMSE for segment 18: 10.1354
RMSE for segment 19: 10.0209
RMSE for segment 20: 9.8053
RMSE for segment 21: 9.6207
RMSE for segment 22: 9.5756
RMSE for segment 23: 9.6144
RMSE for segment 24: 9.8953
RMSE for segment 25: 10.3248
RMSE for segment 26: 10.4398
RMSE for segment 27: 10.6416
RMSE for segment 28: 10.8633
RMSE for segment 29: 11.2793
RMSE for segment 30: 11.3096
RMSE for segment 31: 11.2323
RMSE for segment 32: 11.4933
RMSE for segment 33: 12.0268
RMSE for segment 34: 12.1790
RMSE for segment 35: 11.8756
RMSE for segm

In [43]:
model.save("co_1dcnn.keras")